In [191]:
import pandas as pd
import numpy as np
train_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/train.csv")
test_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/test.csv")
train_titanic.head(),

(  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
 0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
 1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
 2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
 3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
 4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   
 
    RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
 0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
 1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
 2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
 3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
 4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   
 
    Transported  
 0        False  
 1         True  
 2        False  
 3        False  


In [192]:
test_titanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


>+ I shuffle my data with the desire to improve the model's learning ability

In [193]:
from sklearn.utils import shuffle
train_titanic = shuffle(train_titanic)
test_titanic = shuffle(test_titanic)

In [194]:
train_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 6617 to 1155
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 959.3+ KB


In [195]:
train_titanic.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

### **Features scalling data**

In [196]:
col_names = "RoomService", "FoodCourt","ShoppingMall","Spa","VRDeck"
roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = [
    train_titanic.columns.get_loc(c) for c in col_names] # get the column indices
print(roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix)

7 8 9 10 11


In [197]:
import math
from sklearn.base import BaseEstimator, TransformerMixin
attribs_spending_money = [7,8,9,10,11]
# roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = 7,8,9,10,11
class CombineSpendingMoneyOfPerson(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        array = np.zeros(len(X)) # mang mot cot toan so 0
        for ele in range(len(X)):
            for i in attribs_spending_money:
                if ~math.isnan(X[ele][i]):
                    array[ele] += (X[ele][i])
        return np.c_[X,array]

# function for add feature ["spen_money_of_person"]
def feature_scalling_dataset(train, test, drop=None):
    attr_adder = CombineSpendingMoneyOfPerson()
    # Dataset with added columns ["spending_money_of_person"]
    train_titanic_extra_attribs = attr_adder.transform(train.values)
    test_titanic_extra_attribs = attr_adder.transform(test.values)
    train_titanic_extra_attribs = pd.DataFrame(train_titanic_extra_attribs,
                            columns=list(train) + ["spending_money_of_person"],
                            index= train.index)
    test_titanic_extra_attribs = pd.DataFrame(test_titanic_extra_attribs,
                            columns=list(test) + ["spending_money_of_person"],
                            index= test.index)
    return train_titanic_extra_attribs, test_titanic_extra_attribs
train_titanic_1, test_titanic_1 = feature_scalling_dataset(train_titanic, test_titanic)
train_titanic_1.isnull().sum()

PassengerId                   0
HomePlanet                  201
CryoSleep                   217
Cabin                       199
Destination                 182
Age                         179
VIP                         203
RoomService                 181
FoodCourt                   183
ShoppingMall                208
Spa                         183
VRDeck                      188
Name                        200
Transported                   0
spending_money_of_person    908
dtype: int64

In [198]:
# Split columns ["Cabin"] to ["Deck","Side"]
def deck_side_encode(data):
    split_cabin = pd.DataFrame(list(map(lambda x: str(x).split("/"), data.Cabin)))
    new_data = data.copy()
    new_data["Side"] = split_cabin[2]
    new_data["Deck"] = split_cabin[0]
    new_data = new_data.drop("Cabin", axis=1)
    return new_data

In [199]:
attribs_num = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","spending_money_of_person"]
attribs_cat = ["HomePlanet", "Destination", "Cabin"]
attribs_cat_bool = ["CryoSleep", "VIP"]

### **Handle Data**

In [200]:
from sklearn.impute import SimpleImputer
def prepare_data(train, test, drop=None):
    train, test = feature_scalling_dataset(train, test)
    # delete columns Name and PassengerId
    train = train.drop(["Name", "PassengerId"], axis=1)
    test = test.drop(["Name", "PassengerId"], axis=1)
    # missing value
    imputer = SimpleImputer(strategy="mean")
    imputer_cat_bool = SimpleImputer(strategy="most_frequent")
    train[attribs_num] = imputer.fit_transform(train[attribs_num])
    test[attribs_num] = imputer.fit_transform(test[attribs_num])
    train[attribs_cat_bool] = imputer_cat_bool.fit_transform(train[attribs_cat_bool])
    test[attribs_cat_bool] = imputer_cat_bool.fit_transform(test[attribs_cat_bool])
    train[attribs_cat] = imputer_cat_bool.fit_transform(train[attribs_cat])
    test[attribs_cat] = imputer_cat_bool.fit_transform(test[attribs_cat])
    train_titan_encoder = pd.get_dummies(deck_side_encode(train), dummy_na=True, columns=["HomePlanet","Destination","Deck","Side"])
    test_titan_encoder = pd.get_dummies(deck_side_encode(test), dummy_na=True, columns=["HomePlanet","Destination","Deck","Side"])
    # Handle features
    #train data
    train_titan_encoder["CryoSleep"] = train_titan_encoder["CryoSleep"].astype(int)
    train_titan_encoder["VIP"] = train_titan_encoder["VIP"].astype(int)
    train_titan_encoder["Transported"] = train_titan_encoder["Transported"].astype(int)
    # test data
    test_titan_encoder["CryoSleep"] = test_titan_encoder["CryoSleep"].astype(int)
    test_titan_encoder["VIP"] = test_titan_encoder["VIP"].astype(int)
    # test_titan_encoder["Transported"] = test_titan_encoder["Transported"].astype(int)
    return train_titan_encoder, test_titan_encoder

In [201]:
train_prep, test_prep = prepare_data(train_titanic, test_titanic)

In [202]:
train_prep.isnull().sum()

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Transported                  0
spending_money_of_person     0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
HomePlanet_nan               0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Destination_nan              0
Deck_A                       0
Deck_B                       0
Deck_C                       0
Deck_D                       0
Deck_E                       0
Deck_F                       0
Deck_G                       0
Deck_T                       0
Deck_nan                     0
Side_P                       0
Side_S                       0
Side_nan                     0
dtype: int64

In [203]:
train_prep.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,spending_money_of_person,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_nan,Side_P,Side_S,Side_nan
6617,0,41.0,0,0.0,0.0,0.0,0.0,628.000000,0,628.000000,...,0,0,0,0,1,0,0,1,0,0
313,0,22.0,0,142.0,2876.0,202.0,3.0,0.000000,1,3223.000000,...,0,0,0,0,1,0,0,0,1,0
6916,0,22.0,0,0.0,0.0,787.0,1.0,304.854791,0,1484.601541,...,0,0,1,0,0,0,0,1,0,0
8304,0,24.0,0,0.0,784.0,5.0,0.0,1.000000,1,790.000000,...,0,1,0,0,0,0,0,0,1,0
1651,1,59.0,0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,...,0,0,1,0,0,0,0,1,0,0


### **Create and evaluate models**

In [204]:
X_train = np.array(train_prep.drop("Transported", axis=1))
y_train = np.array(train_prep.Transported)
X_test = np.array(test_prep)
X_train.shape, X_test.shape, y_train.shape

((8693, 29), (4277, 29), (8693,))

In [205]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(MinMaxScaler(feature_range=(0,10)), LogisticRegression(random_state=42, solver='lbfgs'))
cross_val_score(pipeline, X_train, y_train, cv=3, scoring="accuracy").mean()

/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

0.7849991015034313

#### Using DecisonTreeClassification


In [206]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {
    'criterion': ["gini","log_loss", "entropy"],
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth':randint(low=2, high=20),
    "min_samples_split":randint(low=1, high = 5),
}

tree_classifier = DecisionTreeClassifier(random_state=42)
tree_grid_search = RandomizedSearchCV(tree_classifier, param_distributions=param_grid,
                                            n_iter=10, cv= 5, scoring="accuracy", random_state=42, verbose=1)

In [207]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_tf = scaler.fit_transform(X_train)
tree_grid_search.fit(X_train_tf, y_train)
# tree_pipeline = make_pipeline(StandardScaler(), tree_grid_search)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 969, in fit
    super().fit(
  File "/home/tuandinh/Desktop/ML

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   param_distributions={'criterion': ['gini', 'log_loss',
                                                      'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb4eba5670>,
                                        'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8,
                                                           9, 10, 11, 12, 13,
                                                           14, 15, 16, 17, 18,
                                                           19, 20, 21, 22, 23,
                                                           24, 25, 26, 27, 28,
                                                           29, 30, 31, ...],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb4ff349a0>},
                   random_state=42, scoring='accuracy', verbose=1)

In [208]:
tree_grid_search.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=16, max_leaf_nodes=63,
                       min_samples_split=3, random_state=42)

In [209]:
tree_grid_search.best_score_

0.7817778824781249

In [210]:
finish_model_decison_tree_classifier = tree_grid_search.best_estimator_

### Evalue model with f1_score

In [211]:
from sklearn.metrics import f1_score
y_pred = finish_model_decison_tree_classifier.predict(X_train_tf)
f1_score(y_train, y_pred)

0.8091707539637276

In [212]:
from sklearn.metrics import classification_report
evalue = classification_report(y_train, y_pred)
print(evalue)

              precision    recall  f1-score   support

           0       0.81      0.80      0.81      4315
           1       0.81      0.81      0.81      4378

    accuracy                           0.81      8693
   macro avg       0.81      0.81      0.81      8693
weighted avg       0.81      0.81      0.81      8693



#### **GradientBoostingClassifier**

In [213]:
from sklearn.ensemble import GradientBoostingClassifier
boosting_param_grid = {
    'loss':["deviance", "exponential"],
    'learning_rate':[0.03, 0.07, 0.1, 0.2],
    'n_estimators': [101, 151, 201, 301, 501, 1000],
    'criterion':["friedman_mse", "squared_error", "mse"],
}

gradient_boosting_classifier = GradientBoostingClassifier()
boosting_grid = RandomizedSearchCV(gradient_boosting_classifier, param_distributions=boosting_param_grid,
                                    n_iter=10, cv= 5, scoring="accuracy", random_state=42, verbose=1)

In [214]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0,5))
pipeline_boosting = make_pipeline(min_max_scaler, boosting_grid)
pipeline_boosting.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWar

Pipeline(steps=[('minmaxscaler', MinMaxScaler(feature_range=(0, 5))),
                ('randomizedsearchcv',
                 RandomizedSearchCV(cv=5,
                                    estimator=GradientBoostingClassifier(),
                                    param_distributions={'criterion': ['friedman_mse',
                                                                       'squared_error',
                                                                       'mse'],
                                                         'learning_rate': [0.03,
                                                                           0.07,
                                                                           0.1,
                                                                           0.2],
                                                         'loss': ['deviance',
                                                                  'exponential'],
                                                         'n_estimators': [101,
                                                                          151,
                                                                          201,
                                                                          301,
                                                                          501,
                                                                          1000]},
                                    random_state=42, scoring='accuracy',
                                    verbose=1))])

In [215]:
boosting_grid.best_estimator_

GradientBoostingClassifier(criterion='mse', learning_rate=0.07,
                           loss='exponential', n_estimators=301)

In [216]:
boosting_grid.best_score_

0.7938560380521059

In [217]:
boosting_model = boosting_grid.best_estimator_

In [219]:
y_pred_boosting = boosting_model.predict(X_train_tf)
error = classification_report(y_train, y_pred_boosting)
print(error)

              precision    recall  f1-score   support

           0       0.87      0.48      0.62      4315
           1       0.64      0.93      0.76      4378

    accuracy                           0.71      8693
   macro avg       0.76      0.70      0.69      8693
weighted avg       0.76      0.71      0.69      8693



In [3]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score

import xgboost as xgb

estimators = [
    ('xgb', xgb.XGBClassifier(max_depth=3, n_estimators=91, subsample=0.65, colsample_bytree=0.65, eval_metric= accuracy_score)),
    ('tree', DecisionTreeClassifier(criterion='entropy', max_depth=16, max_leaf_nodes=63,min_samples_split=3, random_state=42)),
    ('nusvc', make_pipeline(StandardScaler(), NuSVC())),
    ('knn', KNeighborsClassifier(n_neighbors=38))
]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

ModuleNotFoundError: No module named 'xgboost'